In [1]:
# Copyright 2021 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

In this notebook we will use NVIDIA Merlin Models library to build and train two state-of-the-art Deep Learning-based Recommendation models with couple of lines of code:

- Deep Learning Recommendation Model (DLRM)
- Deep & Cross Network (DCN)-V2

For detailed instructions on each building block, and see how you can build the same DLRM model via low-level api you can visit `DLRM_low_level_api_with_prediction.ipynb` notebook.

## Training a DLRM model with TensorFlow

Deep Learning Recommendation Model [(DLRM)](https://arxiv.org/abs/1906.00091) architecture originally proposed by Facebook in 2019. Figure 1 illustrates DLRM architecture. The model was introduced as a personalization deep learning model that uses embeddings to process sparse features that represent categorical data and a multilayer perceptron (MLP) to process dense features, then interacts these features explicitly using the statistical techniques proposed in [here](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=5694074).

![DLRM](../images/DLRM.png)

<p>Figure 1. DLRM architecture. Image source: <a href="https://github.com/NVIDIA/DeepLearningExamples/tree/master/PyTorch/Recommendation/DLRM">Nvidia DL Examples</a></p>

DLRM accepts two types of features: categorical and numerical. For details of the DLRM architecture and how to build it using Merlin Models low-level API please visit `Binary_classificaion_DLRM` notebook.

### Import Libraries

In [2]:
import logging
import os
import glob
import numpy as np
import pandas as pd
import nvtabular as nvt

from merlin.schema import Schema
from merlin.schema.io.tensorflow_metadata import TensorflowMetadata
from merlin.schema.tags import Tags

import merlin_models.tf as mm

2022-02-21 16:39:28.078720: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 16254 MB memory:  -> device: 0, name: Quadro GV100, pci bus id: 0000:15:00.0, compute capability: 7.0


In [3]:
# disable INFO and DEBUG logging everywhere
logging.disable(logging.WARNING) 

### Data Download and Preprocess

In [4]:
INPUT_DATA_DIR = os.environ.get(
    "INPUT_DATA_DIR", os.path.expanduser("/workspace/data/movielens/")
)

With help of a utility function first we download and unzip the data. Second, we convert data via basic preprocessing, and split data into train and validation files and save them as parquet files. Afterwards, we preprocess the train and validation parquet files and generate features for model training using NVTabular.

Let's download Movielens 25M dataset and then process it, and save files to disk in parquet format.

In [5]:
# from merlin_standard_lib.utils.data_etl_utils import movielens_download_etl
# movielens_download_etl(INPUT_DATA_DIR, 'ml-25m')

Merlin Models library relies on a `schema` object to automatically build all necessary layers to represent, normalize and aggregate input features. As you can see below, schema.pb is a protobuf file that contains metadata including statistics about features such as cardinality, min and max values and also tags features based on their characteristics and dtypes (e.g., categorical, continuous, list, integer).

We also generated our `schema.pbtxt` file in using NVTabular. Now we read this schema file to create a `schema` object.

In [6]:
SCHEMA_PATH = os.path.join(INPUT_DATA_DIR, 'ml-25m/train/')
schema = TensorflowMetadata.from_proto_text_file(SCHEMA_PATH).to_merlin_schema()

We can print out the feature names including the binary target column, `rating_binary`, in the schema easily.

In [7]:
schema = schema.without(['rating', 'title'])

In [8]:
schema.column_names

['movieId',
 'userId',
 'genres',
 'TE_movieId_rating',
 'userId_count',
 'rating_binary']

Select continuous and categorical columns from schema using feature tags.

In [9]:
con_schema = schema.select_by_tag(Tags.CONTINUOUS)
cat_schema = schema.select_by_tag(Tags.CATEGORICAL)

In [10]:
con_schema.column_names, cat_schema.column_names

(['TE_movieId_rating', 'userId_count'], ['movieId', 'userId', 'genres'])

### Define Data Loader

Below we define our input block using the `mm.ContinuousEmbedding` function. The from_schema() method processes the schema and creates the necessary layers to represent features and aggregate them.

In [11]:
import merlin_models.tf.dataset as tf_dataloader
def get_dataloader(paths_or_dataset, batch_size=4096, shuffle=True):
    dataloader = tf_dataloader.Dataset(
        paths_or_dataset,
        batch_size=batch_size,
        label_names=['rating_binary'],
        shuffle=shuffle,
        schema = schema,
    )
    return dataloader.map(lambda X, y: (X, tf.reshape(y, (-1,))))

In [12]:
OUTPUT_DIR = os.environ.get("OUTPUT_DIR", "/workspace/data/movielens/ml-25m/")
train_paths = glob.glob(os.path.join(OUTPUT_DIR, "train/*.parquet"))
eval_paths = glob.glob(os.path.join(OUTPUT_DIR, "valid/*.parquet"))

In the DLRM architecture, categorical features are processed using embeddings. Below, for each categorical feature, we create an embedding table used to provide dense representation to each unique value of this feature. The dense vector values in the embedding tables are learned during model training.

### Building a DLRM model with Merlin Models

In [13]:
dlrm_body = mm.DLRMBlock(schema,
        embedding_dim=16,
        bottom_block=mm.MLPBlock([64, 16]),
        top_block=mm.MLPBlock([64, 32]),
    )
model = dlrm_body.connect(mm.BinaryClassificationTask("rating_binary"))

In [14]:
import tensorflow as tf
optimizer = tf.keras.optimizers.Adam(0.005)
model.compile(optimizer=optimizer, run_eagerly=False)

In [15]:
train_loader = get_dataloader(nvt.Dataset(train_paths), shuffle=True) 
eval_loader = get_dataloader(nvt.Dataset(eval_paths), shuffle=False) 

losses = model.fit(train_loader, validation_data=eval_loader, epochs=1)
model.reset_metrics()

/usr/local/lib/python3.8/dist-packages/cudf/core/dataframe.py:1253: UserWarning: The deep parameter is ignored and is only included for pandas compatibility.
  warnings.warn(
2022-02-21 16:39:34.833290: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: annotated name 'output' can't be nonlocal (__autograph_generated_filefizz6obd.py, line 36)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
4883/4883 [==============================] - 94s 17ms/step - rating_binary/binary_classification_task/precision: 0.7625 - rating_binary/binary_classification_task/recall: 0.8606 - rating_binary/binary_classification_task/binary_accuracy: 0.7453 - rating_binary/binary_classification_task/auc: 0.8020 - loss: 0.5143 - regularization_loss: 0.0000e+00 - total_loss: 0.5143 - val_rating_binary/binary_classification_task/precision: 0.7707 - val_rating_binary/binary_classification_task/recall: 0.8651 - val_rating_binary/binary_classification_task/binary_accuracy: 0.7547 - val_rating_binary/binary_classification_task/auc: 0.8163 - val_loss: 0.4835 - 

## Training a Deep & Cross Network (DCN)-V2 model with TensorFlow

[Deep & Cross Network (DCN)-V2](https://arxiv.org/pdf/2008.13535.pdf) architecture was proposed by Google in 2020 as an improve upon the original [DCN model](https://arxiv.org/pdf/1708.05123.pdf). The overall model architecture is depicted in Figure 2, with two ways to combine the cross network with the deep network: (1) stacked and (2) parallel.

![DCN](../images/DCN.png)

<p>Figure 2. DCN-v2 architecture. Image source: <a href="https://arxiv.org/pdf/2008.13535.pdf">DCN V2</a></p>

The output of the embbedding layer is the concatenation of all the embedded vectors and the normalized dense features: x<sub>0</sub> = [x<sub>embed,1</sub>; . . . ; x<sub>embed,𝑛</sub>; x<sub>dense</sub>]. Below, we build a stacked structure shown in Figure 2(a). Basically, it starts with an input layer (typically an embedding layer), and then the input x<sub>0</sub> is fed to the cross network, containing multiple cross layers that models explicit feature interactions, and then followed by the deep network. At the last step, we connect the final layer to the `BinaryClassificationTask` head for doing binary classification.

In [16]:
dcn_body = (
    mm.InputBlock(schema,
        embedding_options=mm.EmbeddingOptions(embedding_dim_default=16),
        aggregation="concat",
    )
    .connect(mm.CrossBlock(3))
    .connect(mm.MLPBlock([512, 256]))
)
model = dcn_body.connect(mm.BinaryClassificationTask("rating_binary"))

In [17]:
optimizer = tf.keras.optimizers.Adam(0.005)
model.compile(optimizer=optimizer, run_eagerly=False)

train_loader = get_dataloader(nvt.Dataset(train_paths), shuffle=True) 
eval_loader = get_dataloader(nvt.Dataset(eval_paths), shuffle=False) 

losses = model.fit(train_loader, validation_data=eval_loader, epochs=1)

/usr/local/lib/python3.8/dist-packages/cudf/core/dataframe.py:1253: UserWarning: The deep parameter is ignored and is only included for pandas compatibility.
  warnings.warn(


4883/4883 [==============================] - 96s 18ms/step - rating_binary/binary_classification_task/precision: 0.7655 - rating_binary/binary_classification_task/recall: 0.8620 - rating_binary/binary_classification_task/binary_accuracy: 0.7487 - rating_binary/binary_classification_task/auc: 0.8068 - loss: 0.5088 - regularization_loss: 0.0000e+00 - total_loss: 0.5088 - val_rating_binary/binary_classification_task/precision: 0.7881 - val_rating_binary/binary_classification_task/recall: 0.8430 - val_rating_binary/binary_classification_task/binary_accuracy: 0.7600 - val_rating_binary/binary_classification_task/auc: 0.8245 - val_loss: 0.4740 - val_regularization_loss: 0.0000e+00 - val_total_loss: 0.4740


Just like that, with couple lines of codes we are able to build state-of-the-art Deep Learning-based Recommender Systems models.